# Coronavirus Data Analysis

In [1]:
%matplotlib notebook

# Module to send http requests
from requests import get

import matplotlib.pyplot as plt
import pandas as pd

from datetime import date
import statistics as stats

In [2]:
df_utlas = pd.read_csv('utlas.csv', names=["Name"])
df_populations = pd.read_csv('populationestimates2020.csv', header=1)
df_utlas = df_utlas.merge(df_populations, how='left', on="Name")

In [3]:
def rolling_average(values, over):
    result = []
    for i in range(over-1):
        result.append(None)
    for i in range(len(values)-over+1):
        result.append(stats.mean(values[i:i+over]))
    return result

def get_data(url):
    response = get(endpoint, timeout=10)
    if response.status_code >= 400:
      raise RuntimeError(f'Request failed: { response.text }')
    return response.json()

In [4]:
def join_on_date(dfs):
    # Takes a dictionary of dataframes
    result = pd.DataFrame()
    for suffix, df in dfs.items():
        if result.empty:
            result = df.rename(columns={f"date{suffix.replace(' ','')}":'date'})
        else:
            result = result.merge(df, left_on='date', right_on=f"date{suffix.replace(' ','')}")
            result['date'].fillna(result[f"date{suffix.replace(' ','')}"], inplace=True)
            del result[f"date{suffix.replace(' ','')}"]
    result['date'] = result['date'].map(date.fromisoformat)
    return result

In [5]:
nations = ['England', 'Scotland', 'Wales', 'Northern Ireland']
populations = {'England':55.268E6, 
               'Scotland': 5.405E6, 
               'Wales': 3.113E6, 
               'Northern Ireland': 1.862E6}


nation_dfs = {}
for nation in nations:
    print(f'processing {nation}')
    endpoint = (
      'https://api.coronavirus.data.gov.uk/v1/data?'
      f'filters=areaType=nation;areaName={nation}&'
      'structure={"date":"date","newCases":"newCasesByPublishDate", "newDeaths":"newDeaths28DaysByPublishDate"}'
    )
    data = get_data(endpoint)
    nation_dfs[nation] = pd.DataFrame(data['data'])\
                                .sort_values('date')\
                                .reset_index(drop=True)\
                                .add_suffix(nation.replace(' ',''))


processing England
processing Scotland


RuntimeError: Request failed: {"response":"An internal error occurred whilst processing your request, please try again. If the problem persists, please report as an issue and include your request.","status_code":500,"status":"Internal Server Error"}

In [ ]:
joined_data_nations = join_on_date(nation_dfs)

In [ ]:
joined_data_nations.to_csv('data_backup.csv')

In [ ]:
for nation in nations:
    joined_data_nations[f"newCasesPerMillion{nation.replace(' ','')}"] = joined_data_nations[f"newCases{nation.replace(' ','')}"]/(populations[nation]/10**6)
    joined_data_nations[f"newDeathsPerMillion{nation.replace(' ','')}"] = joined_data_nations[f"newDeaths{nation.replace(' ','')}"]/(populations[nation]/10**6)
    joined_data_nations[f"newCasesPerMillion7Day{nation.replace(' ','')}"] = rolling_average(joined_data_nations[f"newCasesPerMillion{nation.replace(' ','')}"],7)
    joined_data_nations[f"newDeathsPerMillion7Day{nation.replace(' ','')}"] = rolling_average(joined_data_nations[f"newDeathsPerMillion{nation.replace(' ','')}"],7)

newCasesList = [f"newCasesPerMillion7Day{nation.replace(' ','')}" for nation in nations]
newDeathsList =  [f"newDeathsPerMillion7Day{nation.replace(' ','')}" for nation in nations]

joined_data_nations.plot('date', newCasesList)
plt.legend(labels=nations)
joined_data_nations.plot('date', newDeathsList)
plt.legend(labels=nations)

In [ ]:
utlas = ['Cheshire West and Chester','Leicester','Northumberland','North Yorkshire', 'Wirral', 'Oxfordshire', 'Cumbria']
populations = {'Cheshire West and Chester': 343071,
               'Leicester': 354224,
               'Northumberland': 322434,
               'North Yorkshire': 618054,
               'Wirral': 324011,
               'Oxfordshire': 691667,
               'Cumbria':496200}

mini_df_utlas = df_utlas[utlas]


utla_dfs = {}
for utla in utlas:
    print(f'processing {utla}')
    endpoint = (
      'https://api.coronavirus.data.gov.uk/v1/data?'
      f'filters=areaType=utla;areaName={utla}&'
      'structure={"date":"date","newCases":"newCasesBySpecimenDate", "newDeaths":"newDeaths28DaysByDeathDate"}'
    )
    data = get_data(endpoint)
    utla_dfs[utla] = pd.DataFrame(data['data'])\
                                .sort_values('date')\
                                .reset_index(drop=True)\
                                .add_suffix(utla.replace(' ',''))

In [ ]:
joined_data_utlas = join_on_date(utla_dfs)
joined_data_utlas.tail()

In [ ]:
for utla in utlas:
    joined_data_utlas[f"newCasesPerMillion{utla.replace(' ','')}"] = joined_data_utlas[f"newCases{utla.replace(' ','')}"]/(populations[utla]/10**6)
    joined_data_utlas[f"newCasesPerMillion7Day{utla.replace(' ','')}"] = rolling_average(joined_data_utlas[f"newCasesPerMillion{utla.replace(' ','')}"],7)

newCasesList = [f"newCasesPerMillion7Day{utla.replace(' ','')}" for utla in utlas]

joined_data_utlas.plot('date', newCasesList)
plt.legend(labels=utlas)
plt.title('New Cases per Million Population (7 day rolling)')